# Random forest feature inspection function

## Inspecting feature importance

### Building the Random forest classifier

In [1]:
## Import the random forest model.
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split


def train_rf_model(X_train,y_train, X_test):
    # Random forest model
    rf = RandomForestClassifier(n_estimators=10, random_state=12)

    # Fitting the model
    rf.fit(X_train, y_train)

    # Predicting values
    y_train_pred = rf.predict(X_train)
    y_test_pred = rf.predict(X_test)

    return rf, y_train_pred, y_test_pred    

### Testing with toy dataset from sklearn

In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_wine
data = load_wine()


X = pd.DataFrame(data.data, columns = data.feature_names)
y = pd.DataFrame(data.target)
y = np.array(y)
y = np.ravel(y)

X_train, X_test, y_train, y_test = train_test_split(X,y)

rf, y_train_pred, y_test_pred = train_rf_model(X_train, y_train, X_test)

## Feature importances

### Get the feature importances

In [3]:
def get_feature_importances(trained_model, columns):
    feature_importances = pd.DataFrame(trained_model.feature_importances_,
                                   index = columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
    return feature_importances
    

### One function to call them all

In [4]:
import nbimporter
import performance_measure

def get_rf_model(X_train, X_test, y_train, y_test):
    rf, y_train_pred, y_test_pred = train_rf_model(X_train,y_train, X_test)
    features_imp = get_feature_importances(rf,X_train)
    performance_measure.plot_confusion_matrix(y_test,y_test_pred, normalize=True)
    performance_measure.get_performance_measures(y_test, y_test_pred)
    return features_imp
    

Importing Jupyter notebook from performance_measure.ipynb
Importing Jupyter notebook from pandas_to_latex.ipynb
Importing Jupyter notebook from multiscorer.ipynb


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

In [5]:
rf = get_rf_model(X_train, X_test, y_train,y_test)
rf

ValueError: Shape of passed values is (13, 1), indices imply (133, 1)